# introduction to query execution plans

whenever we write a sql query, the database system first parses our code to generate a query tree.

then the optimizer modifies the query tree either based on relational algebra or cost-based optimization to generate an execution plan.

the execution plan is a sequence of operations that the database system will perform to execute the query.

the operations are things like "scan this table", "filter out rows that don't satisfy this condition", "join these two tables", and so on.

using the `explain` command, we can see the execution plan that the system came up with for our query.

In [41]:
!psql postgres -c "\h explain"

Command:     EXPLAIN
Description: show the execution plan of a statement
Syntax:
EXPLAIN [ ( option [, ...] ) ] statement
EXPLAIN [ ANALYZE ] [ VERBOSE ] statement

where option can be one of:

    ANALYZE [ boolean ]
    VERBOSE [ boolean ]
    COSTS [ boolean ]
    SETTINGS [ boolean ]
    GENERIC_PLAN [ boolean ]
    BUFFERS [ boolean ]
    WAL [ boolean ]
    TIMING [ boolean ]
    SUMMARY [ boolean ]
    FORMAT { TEXT | XML | JSON | YAML }

URL: https://www.postgresql.org/docs/16/sql-explain.html



since the execution plan is a tree, you have to read it from the leaf nodes to the root node.

each child node provides the input to the parent node.

the root node is the final result of the query that aggregates all the intermediate results (and costs).

learn more through the following resources:

_docs_

- https://www.postgresql.org/docs/current/sql-explain.html
- https://www.postgresql.org/docs/9.5/using-explain.html
- overview: https://www.postgresql.org/docs/current/index.html
- performance tips: https://www.postgresql.org/docs/current/performance-tips.html
- glossary: https://www.pgmustard.com/docs/explain

_visualization tools_

- https://explain.dalibo.com/
     - source: https://github.com/dalibo/pev2
- https://www.pgexplain.dev/ (pev2 but with a backend)
     - source: https://github.com/lfborjas/postgres-explain-visualizer
- https://tatiyants.com/pev/
     - source: https://github.com/AlexTatiyants/pev/
- https://explain.depesz.com/
- https://explain-postgresql.com/ (not so good)

# 1) what is the default triangle-join plan?

what is the most common execution plan for the following query: $r \bowtie s \bowtie t$

In [46]:
# reset the environment
!chmod +x ./reset.sh && ./reset.sh >> /dev/null

./reset.sh: line 10: return: can only `return' from a function or sourced script
Did not find any relations.
ERROR:  cannot drop the currently open database
ERROR:  current user cannot be dropped


In [43]:
# print all tables
!psql postgres -c "\dt+"

                                      List of relations
 Schema |    Name     | Type  |  Owner  | Persistence | Access method |  Size   | Description 
--------+-------------+-------+---------+-------------+---------------+---------+-------------
 public | badges      | table | sueszli | permanent   | heap          | 2144 kB | 
 public | comments    | table | sueszli | permanent   | heap          | 8288 kB | 
 public | posthistory | table | sueszli | permanent   | heap          | 31 MB   | 
 public | postlinks   | table | sueszli | permanent   | heap          | 152 kB  | 
 public | posts       | table | sueszli | permanent   | heap          | 15 MB   | 
 public | r           | table | sueszli | permanent   | heap          | 1024 kB | 
 public | s           | table | sueszli | permanent   | heap          | 128 kB  | 
 public | t           | table | sueszli | permanent   | heap          | 1024 kB | 
 public | tags        | table | sueszli | permanent   | heap          | 56 kB   | 
 public

In [44]:
# print schema of r, s, t tables
!psql postgres -c "\d r"
!psql postgres -c "\d s"
!psql postgres -c "\d t"

                 Table "public.r"
 Column |  Type   | Collation | Nullable | Default 
--------+---------+-----------+----------+---------
 a      | integer |           |          | 
 b      | integer |           |          | 

                 Table "public.s"
 Column |  Type   | Collation | Nullable | Default 
--------+---------+-----------+----------+---------
 b      | integer |           |          | 
 c      | integer |           |          | 

                 Table "public.t"
 Column |  Type   | Collation | Nullable | Default 
--------+---------+-----------+----------+---------
 a      | integer |           |          | 
 c      | integer |           |          | 



In [85]:
import os

query = "explain (costs off) SELECT a,b,c FROM r NATURAL JOIN s NATURAL JOIN t;"

last_plan = None
for i in range(20):
    plan = os.popen(f'psql postgres -c "{query}"').read()
    if last_plan is None:
        last_plan = plan
        continue
    assert last_plan == plan, "found different plan"

!psql postgres -c "{query}"

                 QUERY PLAN                  
---------------------------------------------
 Merge Join
   Merge Cond: ((t.c = s.c) AND (t.a = r.a))
   ->  Sort
         Sort Key: t.c, t.a
         ->  Seq Scan on t
   ->  Materialize
         ->  Sort
               Sort Key: s.c, r.a
               ->  Hash Join
                     Hash Cond: (r.b = s.b)
                     ->  Seq Scan on r
                     ->  Hash
                           ->  Seq Scan on s
(13 rows)



without the `analayze` flag, the `explain` command will not actually execute the query and we will always see the same execution plan.

in this case:

In [71]:
import os

query = "SELECT a,b,c FROM r NATURAL JOIN s NATURAL JOIN t;"
full_query = f"explain (analyze, costs off, format json) {query}" # <--- using the analyze option

!psql postgres -c "{full_query}"

last_plan = None
for i in range(20):
    plan = os.popen(f'psql postgres -c "{full_query}"').read()
    if last_plan is None:
        last_plan = plan
        continue
    assert last_plan == plan, "found different plan"

                        QUERY PLAN                         
-----------------------------------------------------------
 [                                                        +
   {                                                      +
     "Plan": {                                            +
       "Node Type": "Merge Join",                         +
       "Parallel Aware": false,                           +
       "Async Capable": false,                            +
       "Join Type": "Inner",                              +
       "Actual Startup Time": 130.294,                    +
       "Actual Total Time": 316.728,                      +
       "Actual Rows": 1969276,                            +
       "Actual Loops": 1,                                 +
       "Inner Unique": false,                             +
       "Merge Cond": "((t.c = s.c) AND (t.a = r.a))",     +
       "Plans": [                                         +
         {                              

AssertionError: found different plan